In [1]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: statsmodels in c:\users\q\appdata\local\programs\python\python38\lib\site-packages (0.14.1)



You should consider upgrading via the 'c:\Users\q\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scikit-learn in c:\users\q\appdata\local\programs\python\python38\lib\site-packages (1.3.2)



You should consider upgrading via the 'c:\Users\q\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [17]:
import pandas as pd

import numpy as np

from statsmodels.formula.api import ols
from statsmodels.api import OLS

from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score, mean_squared_error

from sklearn.model_selection import train_test_split

In [4]:
bikes = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/bikes_sharing.csv')

In [5]:
bikes['datetime'] = pd.to_datetime(bikes['datetime'])

In [7]:
bikes['weekday'] = bikes['datetime'].dt.weekday

In [8]:
day_map = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}

bikes['weekday'] = bikes['weekday'].map(day_map)

bikes

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,weekday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,Sat
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,Sat
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,Sat
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,Sat
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,Sat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,Wed
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,Wed
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,Wed
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,Wed


In [9]:
bikes['hour'] = bikes['datetime'].dt.hour

In [10]:
def get_time_of_day(x):

  if x in range(6,10):

    return 'MorningCommute'

  elif x in range(10,15):

    return 'Midday'

  elif x in range(15,19):

    return 'EveningCommute'

  else:

    return 'Night'

In [12]:
bikes['time_of_day'] = bikes['hour'].apply(get_time_of_day)

In [ ]:
# one hot encode 给很多variable转成dummy variable，取代了很多原本的variable
model_data = pd.get_dummies(bikes, columns=['season','weather','weekday','time_of_day','hour'], drop_first=True, dtype=int)

In [15]:
model_data.columns

Index(['datetime', 'holiday', 'workingday', 'temp', 'atemp', 'humidity',
       'windspeed', 'casual', 'registered', 'count', 'season_2', 'season_3',
       'season_4', 'weather_2', 'weather_3', 'weather_4', 'weekday_Mon',
       'weekday_Sat', 'weekday_Sun', 'weekday_Thu', 'weekday_Tue',
       'weekday_Wed', 'time_of_day_Midday', 'time_of_day_MorningCommute',
       'time_of_day_Night', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'],
      dtype='object')

In [16]:
Y = model_data['count']
#除了'datetime','count', 'casual','registered'，都作为x
X = model_data.drop(columns=['datetime','count', 'casual','registered'])

In [ ]:
model = OLS(Y,X).fit()
print(model.summary())
#有63%可以被这个OLS解释
# 以holiday来说，coef是111，但事实上在95%信赖区间，应该是在96.99~125.8之间
# 如果是weather_4的范围太大，std err很高，p value也太高，表示不显著，也不适合用于预测。但这只是基于目前的dataset,可能因为数据太少，所以不适合预测
# 比如weekday_Thur,这一天的数据，相比于母体数，有很大的差异，可能预测效果不好

                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.633
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     468.2
Date:                Wed, 18 Jun 2025   Prob (F-statistic):               0.00
Time:                        21:30:11   Log-Likelihood:                -66585.
No. Observations:               10886   AIC:                         1.333e+05
Df Residuals:                   10845   BIC:                         1.336e+05
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
holiday             